# Predicting Employee Turnover

## Introduction

Since recruiting, traning and incorporating a new employee in a functioning system of a company is both time-consuming and coslty, a lot of organizations develop different strategy to reduce the employee turnover rate. However, especially in big companies, it is harder to keep track of employees' job satisfaction and willingness to remain in the data by pure communication and gathering *qualitative* data, so collecting quantitative data is often a necessity. Furthermore, collecting data would give us an insight into factors which can determine whether the specific employee will leave the company or not.

The goal of this project was to develop a model which would predict whether an employee will leave a company or not. Apart from that, the goal is to determine which factor drives employee turnover the most! This project would hopefully result in different insights which could become a base fore well-informed strategies aimed at reducing employee churn. With that in mind, let us start exploring dataset (downloaded from Kaggle) with around 14000 employees!

The available features are:
1. satisfaction_level - Satisfaction with job (ranging from 0 to 1)
2. last_evaluation - Years since last evaluation performance
3. number_project - The number of finished projects
4. average_monthly_hours - Average hours spent in workplace per month
5. time_spend_company - Number of years spent in company
6. work_accident - Whether an employee had a work accident (1) or not (0)
7. quit - Whether an employee quit the job (1) or not (0)
8. promotion_last_5years - Whether the employee was promoted in the last five years (1) or not (0)
9. department - Department the employee works in
10. salary - Whether salary is low, medium, or high

In [1]:
# Importing relevant libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.figure_factory as pff
from plotly.subplots import make_subplots
from plotly import graph_objects as go
from pandas_profiling import ProfileReport

In [2]:
pio.renderers.default

'plotly_mimetype+notebook'

In [3]:
# Setting default settings for visualizing data
pio.renderers.default = 'notebook_connected'
pio.templates.default = 'simple_white'

In [4]:
hr = pd.read_csv('employee_data.csv')

In [5]:
# Exploring the features of a dataset
hr.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,quit,promotion_last_5years,department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [6]:
hr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   average_montly_hours   14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  int64  
 6   quit                   14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   department             14999 non-null  object 
 9   salary                 14999 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


We can see that the dataset is pretty much clean since no feature has missing values. In the next part, we are going to examine descriptive statistics for numeric features.

In [7]:
hr.describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,quit,promotion_last_5years
count,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000
mean,0.612834,0.716102,3.803054,201.050337,3.498233,0.144610,0.238083,0.021268
std,0.248631,0.171169,1.232592,49.943099,1.460136,0.351719,0.425924,0.144281
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


Since the quit is our outcome variable and we know that people can either quit their job or not, we expect that the feature is defined through only two levels. By examining the table above, we can see that minimum value for feature *quit* is **0**, while the maximum value for the same feature is **1**. Hence, we can conclude the outcome variable looks fine so far. The further examination of data is presented below.

In [8]:
hr.duplicated().sum()

3008

The code above tested for duplicate rows in the dataset. In fact, there are 3008 same rows in HR dataset. However, since the number of features is small and the scales are highly restricted, it is highly possible that these data actually come from different employees. In ideal situation, we would have an information on the number of employees in the company and then conclude whether these rows are actually duplicate by mistake or not. Since that is not the case in this situation, we are going to assume these rows come from different employees.

In the end, let us make a report on descriptive statistics which summarize the most important results and can be used for communicating data.

In [9]:
# Creating report on descriptive statistics and exporting the file as html
report = ProfileReport(hr, explorative = True)
report.to_file('Report on Descriptive Statistics.html')

# Data Visualization and Exploration

The goal of this section is to visualize univariate distributions of different features. Furthermore, we will also visualize relationship between different features and get the sense which of the features will be the most important for the classification that will come later on.

This section is divided in two parts:
1. Exploring and visualizing non-outcome features
2. Exploring and visualizing relationship between predicting and outcome features

In the first section, we are going to take a look at:
1. Distribution of numerical features
2. Correlations between features

In the second section, we are going to take a look at:
1. Class balance of employees who left their jobs and those who did not
2. Visualizing relationship between outcome features and other features
3. Examining correlations between outcome and other features
4. Define the hypothesis on which features are going to be important for classification

### Exploring and Visualizing Non-Outcome Features

In [10]:
# Preparing iterable variable
histogram_columns = hr.columns[: -5]
rown = [1, 1, 1, 2, 2]
coln = [1, 2, 3, 1, 2]
hist_grid = list(
    zip(
        histogram_columns, rown, coln
    )
)

# Creating multiple histogram subplots
column_names = [
    'Satisfaction Level', 'Last Evaluation', 'Number of Projects', 'Average Monthly Hours', 'Time Spent in Company'
]

histogram_plots = make_subplots(
    rows = 2, cols = 3,
    subplot_titles = column_names
)

for i in hist_grid:
    histogram_plots.add_trace(
        go.Histogram(
            x = hr[i[0]]
        ), 
        row = i[1], col = i[2]
    )

histogram_plots.update_layout(
    showlegend = False, colorway = px.colors.qualitative.D3[:5],
    title = 'Distribution of Numerical Features',
    title_x = 0.5,
    height = 650,
    width = 1100
)

histogram_plots.show()

We can observe that numerical features do not follow the normal distribution. Furthermore, we can see that their scales are different. Therefore, it will be necessary to perform feature scaling before creating the model for predicting employee turnover.

Let us now turn our attention to correlations between these numerical features, so that we can get sense of whether some variables are redundant or nor.

In [11]:
kor = hr.iloc[:, :5].corr().round(3).to_numpy()

heatmap_cor = pff.create_annotated_heatmap(
    kor, 
    x = ['Satisfaction Level', 'Last Evaluation', 'Number of Projects', 'AVG Monthly Hours', 'Time in Company'],
    y = ['Satisfaction Level', 'Last Evaluation', 'Number of Projects', 'AVG Monthly Hours', 'Time in Company'],
    colorscale = 'algae'
)

heatmap_cor.update_layout(
    title = 'Correlations between Numerical Features',
    title_x = 0.5,
    width = 800,
    height = 700
)

heatmap_cor.show()

As a rule of thumb, we can say that correlations stronger than |.7| indicate a presense of multicollinearity. However, the strongest correlation present between these numerical features is .417. Given that the total number of features is not great, we can keep all of the features for developing models.

Now, let us turn our attention to two binary variables - *accidents at work* and *promotion in last five years*. We can simply compute *point-biserial coefficient of correlation* between these features and other numerical features, since both variables have only two levels.

In [12]:
hr.iloc[:, :6].corr(method = 'pearson')['Work_accident'].sort_values()

average_montly_hours   -0.010143
last_evaluation        -0.007104
number_project         -0.004741
time_spend_company      0.002120
satisfaction_level      0.058697
Work_accident           1.000000
Name: Work_accident, dtype: float64

In [13]:
columns_for_corr = [
    'satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'promotion_last_5years'
]

hr.loc[:, columns_for_corr].corr(method = 'pearson')['promotion_last_5years'].sort_values()

last_evaluation         -0.008684
number_project          -0.006064
average_montly_hours    -0.003544
satisfaction_level       0.025605
time_spend_company       0.067433
promotion_last_5years    1.000000
Name: promotion_last_5years, dtype: float64

By examining last two outputs, we can observe that the relationship between fetures *'promotion_last_5years'*/*'Work_accident'* and other numerical features are low, with strongest correlation being *0.067*. Therefore, we can conclude these features are not redundant and may even explain additional variance of our outcome variable that cannot be explained by other features!

### Exploring and Visualizing Relationship Between Predicting and Outcome Features

In [14]:
hr['quit_describe'] = 'Quit'
hr.loc[hr['quit'] == 0, 'quit_describe'] = "Did not quit"

In [15]:
hr.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,quit,promotion_last_5years,department,salary,quit_describe
0,0.38,0.53,2,157,3,0,1,0,sales,low,Quit
1,0.80,0.86,5,262,6,0,1,0,sales,medium,Quit
2,0.11,0.88,7,272,4,0,1,0,sales,medium,Quit
3,0.72,0.87,5,223,5,0,1,0,sales,low,Quit
4,0.37,0.52,2,159,3,0,1,0,sales,low,Quit


In [16]:
quit_graph = px.histogram(
    data_frame = hr, 
    x = 'quit_describe', 
    orientation = 'v', 
    histfunc = 'count', 
    color = 'quit_describe',
    color_discrete_sequence = ['#D62728', '#2CA02C']
)

quit_graph.update_layout(
    title = 'The Number of Employees who Quit',
    title_x = 0.5,
    xaxis_title = 'Quit or Not',
    yaxis_title = 'Total Count',
    width = 600,
    height = 500
)

quit_graph.show()

Now, let us see the distribution of different numerical features depending on the fact that the employee has either quit the job or not. Since the numeric features do not have the same scale (some range from 0 to 1, while some have greater range), we can either standardize data or create multiple subplots.

We are going to create multiple subplots in this case in order to get more sense of data by examining their original values and ranges.

In [17]:
columns_for_box = [
    'satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company'
]

row = [1, 1, 2, 2, 3]
col = [1, 2, 1, 2, 1]
colors = [
    'rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)',
          'rgba(255, 65, 54, 0.5)', 'rgba(207, 114, 255, 0.5)'
]

grid_list = list(
    zip(
        columns_for_box, row, col, colors
    )
)

box_plots = make_subplots(
    rows = 3, cols = 2,
    subplot_titles = [
        'Satisfaction Level', 'Last Evaluation', 'Number of Projects', 'Average Monthly Hours', 'Time Spent in Company'
    ],
    x_title = 'Quit or not',
    y_title = 'Values'
)

for i in grid_list:
    box_plots.add_trace(
        go.Box(
            y = hr[i[0]], 
            x = hr['quit_describe'], 
            boxmean = 'sd', 
            fillcolor = i[3], 
            jitter = 0.4, 
            whiskerwidth = 0.2, 
            marker_size = 2
        ),
        row = i[1], col = i[2]
    )

box_plots.update_layout(
    title='Numerical Features and Outcome Feature',
    title_x = 0.5,
    height = 1100,
    width = 1100,
    showlegend = False
)

box_plots.show()

By looking at boxplots shown above, we can see that there is higher variation for most of numerical features when it comes to those who quit in comparison to those who did not leave the company. Now, let us examine the matrix correlation between different variables and see which features will be promising in classifying workers in those who left the company and those who stayed in the company.

Even though the *Quit* column is categorical column, we can still calculate *Point-biserial coefficients of correlations* (i.e. *Pearson product moment correlation*) since the *Quit* columns has only two levels!

In [18]:
# Correlations from negative to positive
corr_direction = hr.corr(
    method = 'pearson')['quit'].sort_values()

print(corr_direction)

satisfaction_level      -0.388375
Work_accident           -0.154622
promotion_last_5years   -0.061788
last_evaluation          0.006567
number_project           0.023787
average_montly_hours     0.071287
time_spend_company       0.144822
quit                     1.000000
Name: quit, dtype: float64


In [19]:
# Absolute strength of correlation
strength = np.abs(
    hr.corr(
        method = 'pearson')['quit']
).sort_values()

print(strength)

last_evaluation          0.006567
number_project           0.023787
promotion_last_5years    0.061788
average_montly_hours     0.071287
time_spend_company       0.144822
Work_accident            0.154622
satisfaction_level       0.388375
quit                     1.000000
Name: quit, dtype: float64


Now, let us visualize the correlations strength!

In [20]:
correlations = px.bar(
    y = strength, x = strength.index, color = strength.index, 
    color_discrete_sequence = px.colors.sequential.Greens[:8]
)
                                                 
correlations.update_layout(
    title = 'Absolute Strength of Correlation between Quit and Other Numerical Features',
    title_x = 0.5,
    width = 800,
    height = 500,
    yaxis_title = 'Value',
    xaxis_title = 'Feature',
    showlegend = False
)

correlations.show()

By examining the graph above, we can already see that the correlations are of low-to-medium strength. Furthermore, we can see that satisfaction level rises up as one of the potentially most important features for discriminating between employees who left their jobs and employees who did not. That observation is somewhat expected - those who are not satisfied with their jobs are more likely to quit (referenca).

After we have examined correlations between *Quit* feature and other numerical features, let us now turn our attention to four remaining categorical features - *Job Accidents*,*Job Promotions*, *Salary* and *Department*. Since these features have multiple levels, calculating correlations would require creating dummy variables. Therefore, we are going to present data visually.

THIS PART NEEDS TO BE FINISHED.

# Data Preprocessing

Let us quickly remind ourselves that not all features are in the adequate shape for further model development. In other words, some features actually consist of multiple categories and dummy coding is necessary.

In [21]:
print(hr['salary'].unique())

['low' 'medium' 'high']


In [22]:
print(hr['department'].unique())

['sales' 'accounting' 'hr' 'technical' 'support' 'management' 'IT'
 'product_mng' 'marketing' 'RandD']


In [23]:
hr_with_first = pd.get_dummies(hr, columns = ['salary', 'department'], drop_first = False)
hr_without_first = pd.get_dummies(hr, columns = ['salary', 'department'], drop_first = True)

In [24]:
# Preparing train and test data
from sklearn.model_selection import train_test_split
X = hr_without_first.loc[:, hr_without_first.columns != 'quit']
X = X.drop(['quit_describe'], axis = 1)
y = hr_without_first.loc[:, 'quit']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 123)

In [25]:
# Feature scaling
from sklearn.preprocessing import StandardScaler
stdscaler = StandardScaler(with_mean = True, with_std = True)
X_train_std = X_train

# We are going to standardize numerical features only
X_train_std.iloc[:, :5] = stdscaler.fit_transform(X_train.iloc[:, :5])
X_train_std.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,salary_low,salary_medium,department_RandD,department_accounting,department_hr,department_management,department_marketing,department_product_mng,department_sales,department_support,department_technical
6197,-1.862862,-1.900843,0.97280,0.563659,0.344888,0,0,1,0,0,0,0,0,0,0,1,0,0
1955,-2.104377,0.555873,1.78594,1.124295,0.344888,0,0,1,0,0,0,0,0,0,0,0,1,0
9440,-0.172258,-1.315910,-1.46662,0.723841,-1.027914,0,0,1,0,0,0,0,0,0,0,0,1,0
9365,0.954811,-0.730978,-1.46662,1.264454,-1.027914,0,0,1,0,0,0,0,0,0,0,0,1,0
7725,-1.822610,0.029434,0.15966,-0.177181,1.717690,0,0,0,0,0,0,0,0,0,0,1,0,0


# Model Development

After we have prepared data for further model development, we have to choose a few algorithms with which we are going to try and develop the model for classifying our employees. Since this is a problem of classification, we could try some of the following algorithms:
1. Logistic Regression
2. Linear Support Vector Classifier
3. Kernelised Support Vector Classifier
4. Decision Tree Classifier

All of these models will be cross-validated with 10-k folds. These models will be developed with few-to-none hyptermarameter tuning, so that we can list a few promising models. After we get up to three promising models, the next section will cover hyperparameter tuning, while the final section will cover developing more complex ensemble methods, such as Random Forest Classifier or applying Adaptive Boosting.

## Choosing Evaluation Metrics

Since the task at hand is to predict whether employee will leave the job or not, we have to choose specific evaluation metrics in order to evaluate the strength of the model. Since we have class inbalance, we should opt for AUC score as one of the most important metrics for evaluating models, although we are going to use accuracy, precision and recall scores as well!

In [26]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

## Training Logistic Regression

### Unconstrained Logistic Regression

In [27]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(
    penalty = 'none', 
    n_jobs = -1, 
    solver = 'lbfgs',
    class_weight = 'balanced',
    random_state = 123
)

lr.fit(
    X_train_std, y_train
)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=-1, penalty='none',
                   random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [28]:
lr_ac = accuracy_score(y_train, lr.predict(X_train_std))
lr_pr = precision_score(y_train, lr.predict(X_train_std))
lr_re = recall_score(y_train, lr.predict(X_train_std))
lr_auc = roc_auc_score(y_train, lr.predict_proba(X_train_std)[:, 1])

log_metrics_noncv = pd.Series(
    {'Accuracy score' :lr_ac,
     'Precision score' : lr_pr, 
     'Recall score' : lr_re, 
     'AUC score' : lr_auc}
)

log_metrics_noncv

Accuracy score     0.759647
Precision score    0.497057
Recall score       0.798040
AUC score          0.830463
dtype: float64

Since this model is significantly underperforming, let us first cross validate it and then try using polynomial features.

In [29]:
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate
c_lr = cross_validate(
    lr, 
    X_train_std,
    y_train, 
    cv = 10, 
    scoring = [
        'accuracy', 'precision', 'recall', 'roc_auc'
    ],
    n_jobs = -1
)

log_metrics_cv = pd.DataFrame(c_lr).mean()[2:]
log_metrics_cv

test_accuracy     0.758400
test_precision    0.495703
test_recall       0.794893
test_roc_auc      0.829185
dtype: float64

#### Logistic Regression: Final Results from Cross-Validation without Using Polynomial Features

| Metric  | Value|
|-------- |------|
|Accuracy |0.758 |
|Precision|0.496 |
|Recall   |0.795 |
|AUC      |0.829 |

In the end, let us plot the receiver operating characteristic (ROC) curve, which plots True Positive Rate against False Positive Rate. In general, we would want the True Positive Rate to dramatically increase with a little-to-no increase in False Positive Rate.

In [30]:
# Saving False Positive Rates and True Positive Rates for different thresholds
from sklearn.metrics import roc_curve
y_scores = cross_val_predict(
    lr, 
    X_train_std, 
    y_train,
    cv = 10, 
    method = 'decision_function', 
    n_jobs = -1)
fpr_lr, tpr_lr, thresholds_lr = roc_curve(y_train, y_scores)

In [31]:
# Defining the function for plotting the roc curve
def roc_plot(fpr, tpr, title):
    roc_lr_poly = px.line(
        x = fpr, 
        y = tpr, 
        labels = {'x' : "False Positive Rate", 'y' : "True Positive Rate"}
    )
    roc_lr_poly.add_shape(
        type = 'line', 
        x0 = 0, 
        y0 = 0, 
        x1 = 1, 
        y1 = 1, 
        line_color = 'Grey',
        line_width = 3,
        line_dash = 'dashdot'
    )
    roc_lr_poly.update_layout(title=title,
        title_x = 0.5,
        height = 600,
        width = 700)
    roc_lr_poly.show()

In [32]:
roc_plot(
    fpr_lr, 
    tpr_lr, 
    'ROC curve for Logistic Regression'
)

We can see that the model is not underperforming while using cross validation. Hence, we should try a bit more complicated models. 

### Polynomial Logistic Regression

In [33]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(
    degree = 3
)

X_train_poly = X_train
X_train_poly = poly.fit_transform(X_train_poly.iloc[:, :5])
X_train_poly = stdscaler.fit_transform(X_train_poly)
X_train_poly = np.hstack([X_train_poly, X_train.iloc[:, 5:]])

In [34]:
lr.fit(
    X_train_poly, y_train
)

print(
    'Accuracy:', accuracy_score(
        y_train, lr.predict(X_train_poly)
    )
)

Accuracy: 0.9459121593466122


By examining the resulting accuracy above, it can be seen that increasing polynomial features for the whole training set improves accuracy significantly. Before jumping to any conclusions, the model should be cross-validated and further metrics should be examined.

In [35]:
c_lr_poly = cross_validate(
    lr, 
    X_train_poly, 
    y_train,     
    scoring = [
        'accuracy', 'precision', 'recall', 'roc_auc'
    ],
    n_jobs = -1,
    cv = 10
)

In [36]:
log_poly_metrics_cv = pd.DataFrame(c_lr_poly).mean()[2:]
log_poly_metrics_cv

test_accuracy     0.943329
test_precision    0.844608
test_recall       0.935242
test_roc_auc      0.979384
dtype: float64

#### Logistic Regression: Final Results from Cross-Validation With Polynomial Features

| Metric  | Value|
|-------- |------|
|Accuracy |0.943 |
|Precision|0.845 |
|Recall   |0.935 |
|AUC      |0.979 |

In [37]:
y_scores = cross_val_predict(
    lr, 
    X_train_poly, 
    y_train, 
    cv = 10, 
    method = 'decision_function', 
    n_jobs = -1
)
fpr_poly, tpr_poly, thresholds_poly = roc_curve(y_train, y_scores)

In [38]:
roc_plot(
    fpr_poly, 
    tpr_poly, 
    'ROC curve for Logistic Regression With Polynomial Features'
)

### Model Summary
In the previous section, Logistic Regression model was trained without any constraints. The metrics used for evaluating the model's performance showed that is significanly underfit the data - precision is lower than **0.5**. Since that was the case, the next step was to train Polynomial Logistic Regression model; in other words, the more complicated model is required in order to capture the relationship between predicting features and outcome feature.

The metrics used for evaluating model's performance indicate that Polynomial Logistic Regression is a promising model - its performance is significantly better that the performance of unconstrained Logistic Regression model. Hence, Polynomial Logistic Regression seems to be a promising model for predicting employee turnover, even without much hyperparameter (degree of polynomial) tuning!

Since the improvement of the model indicates the relatinoship between predicting features and outcome feature is nonlinear, we should explore some other models which are better and comuptationally more efficient at capturing the nonlinear relationship (e.g. Support Vector Classifier with Polynomial Kernel).

## Training Linear SVC

In [39]:
from sklearn.svm import LinearSVC
linSVC = LinearSVC(
    max_iter = 50000, 
    C = 10, 
    class_weight = 'balanced', 
    random_state = 123
)

In [40]:
linsvc_cv = cross_validate(
    linSVC,
    X_train_std,
    y_train,
    scoring = [
        'accuracy', 'precision', 'recall', 'roc_auc'
    ],
    cv = 10,
    n_jobs = -1
)

In [41]:
linsvc_metrics_cv = pd.DataFrame(linsvc_cv).mean()[2:]
linsvc_metrics_cv

test_accuracy     0.753233
test_precision    0.488924
test_recall       0.784401
test_roc_auc      0.828340
dtype: float64

### Linear Support Vector Classifier: Final Results from Cross-Validation

| Metric  | Value|
|-------- |------|
|Accuracy |0.753 |
|Precision|0.489 |
|Recall   |0.784 |
|AUC      |0.828 |

In [42]:
y_scores = cross_val_predict(
    linSVC, 
    X_train_std, 
    y_train, 
    cv = 10, 
    n_jobs = -1, 
    method = 'decision_function'
)
fpr_linsvc, tpr_linsvc, thresholds_linsvc = roc_curve(y_train, y_scores)

In [43]:
roc_plot(
    fpr_linsvc, 
    tpr_linsvc, 
    'ROC Curve for Linear Support Vectors Classifier'
)

### Model Summary
The results obtained by developing unconstrained Logistic Regression model indicated that using linear classifiers for capturing the relationship between predicting features and outcome feature is probably not the best strategy. Results from Linear Support Vector Classifier confirms that assumption. Therefore, Linear SVC does not seem to be a promising model. With that in mind, more complex model should be developed, such as SVC with Polynomial or RBF Kernel.

## Training Kernelised SVC

### Training SVC with Polynomial Kernel

In [44]:
from sklearn.svm import SVC

In [45]:
svc_poly = SVC(
    kernel = 'poly', 
    degree = 3, 
    random_state = 123, 
    class_weight = 'balanced'
)
svc_poly_cv = cross_validate(
    svc_poly, 
    X_train_std, 
    y_train, 
    n_jobs = -1, 
    scoring = [
        'accuracy', 'precision', 'recall', 'roc_auc'
    ],
    cv = 10
)

In [46]:
svc_poly_metrics = pd.DataFrame(svc_poly_cv).iloc[:, 2:].mean()
svc_poly_metrics

test_accuracy     0.945412
test_precision    0.864066
test_recall       0.915639
test_roc_auc      0.966675
dtype: float64

#### Support Vector Classifier with Polynomial Kernel: Final Results from Cross-Validation

| Metric  | Value|
|-------- |------|
|Accuracy |0.945 |
|Precision|0.864 |
|Recall   |0.916 |
|AUC      |0.967 |

In [47]:
y_scores = cross_val_predict(
    svc_poly, 
    X_train_std, 
    y_train, 
    cv = 10, 
    n_jobs = -1, 
    method = 'decision_function'
)
fpr_svc_poly, tpr_svc_poly, thresholds_svc_poly = roc_curve(y_train, y_scores)

In [48]:
roc_plot(fpr_svc_poly, tpr_svc_poly, 'ROC Curve for SVC with Polynomial Kernel')

#### Model Summary
Developing SVC with Polynomial Kernel resulted in really high accuracy, precision, recall and area under the curve measuers! These results were expected since Polynomial Logistic Regression already showed that the relationship between predicting features and outcome feature is nonlinear. Furthermore, SVC with Polynomial Kernel seems to be a promising strategy since good metrics were observed with no hyperparameter tuning. In the end, SVC with Polynomial Kernel is computationally more efficient than developing High-degree Polynomial Logistic Regression models. Therefore, the hyperparameters of this model are going to be tuned in the next section, while Polynomial Logistic Regression model will be excluded from the list of promising models.

Even though SVC with Polynomial Kernel is more computationally efficient than SVC with RBF Kernel when there is a large number of training instances, we will try and explore it nonetheless. We would expect that the *area under the curve* value is going to be larger. However, if only slight improvement is observed, SVC with RBF Kernel will not be explored any further, since the time needed to train is going to be much longer.

### Training SVC with RBF Kernel

In [49]:
svc = SVC(gamma = 2, class_weight = 'balanced', random_state = 123)
svc_cv = cross_validate(
    svc, 
    X_train_std, 
    y_train, 
    n_jobs = -1, 
    scoring = [
        'accuracy', 'precision', 'recall', 'roc_auc'
    ], 
    cv = 10
)
svc_cv_metrics = pd.DataFrame(svc_cv).iloc[:, 2:].mean()
svc_cv_metrics

test_accuracy     0.980081
test_precision    0.974663
test_recall       0.940831
test_roc_auc      0.988709
dtype: float64

#### Support Vector Classifier with RBF Kernel: Final Results from Cross-Validation

| Metric  | Value|
|-------- |------|
|Accuracy |0.980 |
|Precision|0.975 |
|Recall   |0.941 |
|AUC      |0.989 |

In [50]:
y_scores = cross_val_predict(
    svc, 
    X_train_std, 
    y_train, 
    n_jobs = -1, 
    method = 'decision_function', 
    cv = 10
)

In [51]:
fpr_svc, tpr_svc, thresholds_svc = roc_curve(y_train, y_scores)
roc_plot(
    fpr_svc, 
    tpr_svc, 
    'ROC Curve for SVC with RBF Kernel'
)

#### Model Summary
Since SVC with RBF Kernel is a great algorithm for capturing complex nonlinear relationships, it should come as no surprise that the model performance as evaluated by chosen metrics is better when compared to e.g. Linear SVC or Logistic Regression models! Furthermore, the performance is slightly better than the performance of SVC with Polynomial Kernel. That should have also been expected, since SVC with RBF Kernel creates new features which number is equal to the number of training instances (i.e. around 12.000). Therefore, even RBF Kernel leads to slighty better results, Polynomial Kernel is computationally more efficient!

In the end, it can be concluded that jyperparameters of both SVC with Polynomial Kernel and SVC with RBF Kernel should be tuned in the next section, since both models seem to be a good strategy at capturing the relationship between predicting features and outcome feature!

## Training Decision Tree Classifier

In [52]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
dct = DecisionTreeClassifier(
    max_depth = 20, 
    random_state = 123, 
    class_weight = 'balanced'
)

In [53]:
c_dct = cross_validate(
    dct, 
    X_train_std, 
    y_train, 
    cv = 10, 
    scoring = [
        'accuracy', 'precision', 'recall', 'roc_auc'
    ], 
    n_jobs = -1
)
pd.DataFrame(c_dct).iloc[:, 2:].mean()

test_accuracy     0.978164
test_precision    0.946517
test_recall       0.962892
test_roc_auc      0.972914
dtype: float64

### Decision Tree Classifier: Final Results from Cross-Validation

| Metric  | Value|
|-------- |------|
|Accuracy |0.978 |
|Precision|0.947 |
|Recall   |0.963 |
|AUC      |0.973 |

In [54]:
y_scores = cross_val_predict(
    dct, 
    X_train_std, 
    y_train, 
    cv = 10, 
    method = 'predict_proba', 
    n_jobs = -1
)
y_scores = y_scores[:, 1]

In [55]:
fpr_dct, tpr_dct, thresholds_dct = roc_curve(y_train, y_scores)
roc_plot(
    fpr_dct, 
    tpr_dct, 'ROC Curve for Decition Tree Classifier'
)

### Model Summary
Decision Tree Classifier resulted in high evaluation metrics, even with constraining the maximum depth of tree to 20 and no hyperparameter tuning! Since it seems that Decision Tree Classifier is a promising model, its hyperparameters actually won't be tuned in the next section, since much better strategy is to develop Random Forest Classifier and than tune its parameters!

## Brief Summary of All Models

A few different models were developed in the previous section with a goal of predicting the employee turnover. Performance of each individual model was presented and plotted separately. However, it is useful to plot ROC curves of each model on the same plot. The said plot is presented below.

In [56]:
roc_curve_plot = go.Figure()
roc_curve_plot.add_trace(
    go.Scatter(
        x = fpr_lr, y = tpr_lr, name = "Logistic Regression")
)
roc_curve_plot.add_trace(
    go.Scatter(
        x = fpr_dct, y = tpr_dct, name = 'Decision Tree Classifier', mode = 'lines')
)
roc_curve_plot.add_trace(
    go.Scatter(
        x = fpr_svc, y = tpr_svc, name = 'SVC with RBF Kernel')
)
roc_curve_plot.add_trace(
    go.Scatter(
        x = fpr_svc_poly, y = tpr_svc_poly, name = 'SVC with Polynomial Kernel')
)
roc_curve_plot.add_trace(
    go.Scatter(
        x = fpr_poly, y = tpr_poly, name = 'Polynomial Logistic Regression')
)
roc_curve_plot.add_trace(
    go.Scatter(
        x = fpr_linsvc, y = tpr_linsvc, name = 'Linear SVC')
)
roc_curve_plot.add_shape(
        type = 'line', 
        x0 = 0, 
        y0 = 0, 
        x1 = 1, 
        y1 = 1, 
        line_color = 'Grey',
        line_width = 3,
        line_dash = 'dashdot'
    )
roc_curve_plot.update_layout(
    title = 'ROC Curves for Different Models', 
    title_x = 0.5, 
    xaxis_title = 'False Positive Rate',
    yaxis_title = 'True Positive Rate', 
    height = 500, 
    width = 700
)

By examining the plot above, it can concluded that two models are significantly worse than the others, and those are Logistic Regression and Linear Support Vector Classifier. Hence, those models will not be examined any further.

Let us now turn our attention to the models which perform really well - Polynomial Logistic Regression, SVC with RBF Kernel, SVC with Polynomial Kernel and Decision Tree Classifier. Polynomial Logistic Regression and SVC with Polynomial Kernel have similar results when it comes to *area under the curve* metric. Since SVC with Polynomial Kernel is computationally more efficient than Polynomial Logistic Regression, Polynomial Logistic Regression will be excluded as well!

The remaining three models are Decision Tree Classifier, SVC with RBF Kernel and SVC with Polynomial Kernel. Even though SVC with RBF Kernel resulted in higher *AUC* measure than SVC with Polynomial Kernel, there are several arguments to be considered before tuning hyperparameters for SVC with RBF Kernel. First of all, much simpler model (Decision Tree Classifier) resulted in similar metrics, while the training time was considerably shorter. Second of all, SVC with RBF Kernel is comuptationally more expensive, so the trade-off between metrics and efficiency of the model is something that needs to be considered. Lastly, if tuning SVC with Polynomial Kernel leads to similar results as SVC with RBF Kernel has yielded, than tuning the second one won't be necessary since we already have pretty accurate predictions, and improvement by the third decimal would not be a significant improvement. 

As for the Decision Tree Classifier, we are not going to tune the parameters of a single Decision Tree, since developing a Random Forest Classifier will yield even better results.

# Hyperparameter Tuning for SVC with Polynomial Kernel

In [57]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [58]:
svc_poly_params = {
    'C' : [3, 5, 8, 10, 12, 15, 18], 
    'degree' : [2, 3, 4], 
    'coef0' : [0.01, 0.05, 0.1]
}

In [59]:
grid_poly_svc= GridSearchCV(
    estimator = SVC(
        random_state = 123, 
        class_weight = 'balanced'
    ), 
    param_grid = svc_poly_params,
    n_jobs = -1, 
    cv = 10, 
    scoring = ['roc_auc', 'accuracy', 'precision', 'recall'],
    refit = 'roc_auc'
)

In [60]:
grid_poly_svc.fit(X_train_std, y_train)

C:\Users\sdgem\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.



GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight='balanced',
                           coef0=0.0, decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=123, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': [3, 5, 8, 10, 12, 15, 18],
                         'coef0': [0.01, 0.05, 0.1], 'degree': [2, 3, 4]},
             pre_dispatch='2*n_jobs', refit='roc_auc', return_train_score=False,
             scoring=['roc_auc', 'accuracy', 'precision', 'recall'], verbose=0)

In [61]:
print(grid_poly_svc.best_estimator_)
grid_poly_svc.best_score_

SVC(C=18, cache_size=200, class_weight='balanced', coef0=0.01,
    decision_function_shape='ovr', degree=2, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=123,
    shrinking=True, tol=0.001, verbose=False)


0.9803173384149978

In [62]:
grid_poly_svc.best_params_

{'C': 18, 'coef0': 0.01, 'degree': 2}

In [63]:
svc_poly_metrics = pd.DataFrame(grid_poly_svc.cv_results_)
svc_poly_metrics.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_C', 'param_coef0', 'param_degree', 'params',
       'split0_test_roc_auc', 'split1_test_roc_auc', 'split2_test_roc_auc',
       'split3_test_roc_auc', 'split4_test_roc_auc', 'split5_test_roc_auc',
       'split6_test_roc_auc', 'split7_test_roc_auc', 'split8_test_roc_auc',
       'split9_test_roc_auc', 'mean_test_roc_auc', 'std_test_roc_auc',
       'rank_test_roc_auc', 'split0_test_accuracy', 'split1_test_accuracy',
       'split2_test_accuracy', 'split3_test_accuracy', 'split4_test_accuracy',
       'split5_test_accuracy', 'split6_test_accuracy', 'split7_test_accuracy',
       'split8_test_accuracy', 'split9_test_accuracy', 'mean_test_accuracy',
       'std_test_accuracy', 'rank_test_accuracy', 'split0_test_precision',
       'split1_test_precision', 'split2_test_precision',
       'split3_test_precision', 'split4_test_precision',
       'split5_test_precision', 'split6_test_precision',
       '

In [64]:
svc_poly_metrics = svc_poly_metrics[
    ['param_C', 'param_coef0', 'param_degree', 'params', 'mean_test_roc_auc', 'mean_test_accuracy', 'mean_test_precision', 'mean_test_recall']]
svc_poly_metrics['mean_metrics'] = svc_poly_metrics.iloc[:, 4:].mean(axis = 1)
svc_poly_metrics.head(3)

,param_C,param_coef0,param_degree,params,mean_test_roc_auc,mean_test_accuracy,mean_test_precision,mean_test_recall,mean_metrics
0,3,0.01,2,"{'C': 3, 'coef0': 0.01, 'degree': 2}",0.978191,0.949329,0.867948,0.929642,0.931278
1,3,0.01,3,"{'C': 3, 'coef0': 0.01, 'degree': 3}",0.978191,0.949329,0.867948,0.929642,0.931278
2,3,0.01,4,"{'C': 3, 'coef0': 0.01, 'degree': 4}",0.978191,0.949329,0.867948,0.929642,0.931278


In [65]:
svc_poly_metrics.loc[svc_poly_metrics['params'] == grid_poly_svc.best_params_, :].iloc[:, 4:].T

,54
mean_test_roc_auc,0.980317
mean_test_accuracy,0.960330
mean_test_precision,0.898849
mean_test_recall,0.939793
mean_metrics,0.944822


In [66]:
mean_metrics_poly = px.line(
    data_frame = svc_poly_metrics, 
    x = svc_poly_metrics.index, 
    y = 'mean_metrics'
)
mean_metrics_poly.update_layout(
    title = 'Increase in Mean Metrics with Hyperparameter Tuning',
    title_x = 0.5,
    xaxis_title = 'Index of Hyperparameters',
    yaxis_title = 'Mean Metrics',
    height = 500,
    width = 700
)
mean_metrics_poly.show()

By observing the graph above, we can see that with every few models trained, there is a step increase in Mean Metrics. Since the first hyperparameter that was tuned was **C** regularization parameter, which means that for each C, Grid Search tested every other hyperparameter. That leads us to conclusion that the final step increase occurs when C hyperparameter is the highest (18), while other hyperparameters do not have much impact on mean metrics increase in that situation. 

However, we can see that the increase actually in mean metrics is actually really low with increase in C regularization parameter (i.e. the third decimal changes). Therefore, it could be better not to tune C regularizatio parameter any more, since increacing C parameter may lead to worse generalization. Hence, we should keep this estimator as the final SVC with Polynomial Kernel estimator.

The results from untuned SVC with Polynomial Kernel were:

| Metric  | Value|
|-------- |------|
|Accuracy |0.945 |
|Precision|0.864 |
|Recall   |0.916 |
|AUC      |0.967 |

On the other hand, retuls from tuned SVC with Polynomial Kernel are:

| Metric  | Value|
|-------- |------|
|Accuracy |0.960 |
|Precision|0.899 |
|Recall   |0.940 |
|AUC      |0.980 |

In the end, let us compare the ROC curves of untuned and tuned SVC with Polynomial Kernel models.

In [67]:
y_scores = cross_val_predict(
    grid_poly_svc.best_estimator_, 
    X_train_std, 
    y_train, 
    cv = 10, 
    n_jobs = -1, 
    method = 'decision_function'
)
fpr_svc_poly_t, tpr_svc_poly_t, thresholds_svc_poly_t = roc_curve(y_train, y_scores)

In [68]:
roc_svc_poly_tuned = go.Figure()
roc_svc_poly_tuned.add_trace(
    go.Scatter(
        x = fpr_svc_poly, 
        y = tpr_svc_poly, 
        name = 'Untuned SVC with Poly Kernel'
    )
)
roc_svc_poly_tuned.add_trace(
    go.Scatter(
        x = fpr_svc_poly_t, 
        y = tpr_svc_poly_t, 
        name = 'Tuned SVC with Poly Kernel'
    
    )
)
roc_svc_poly_tuned.add_trace(
    go.Scatter(
        x = fpr_svc, 
        y = tpr_svc, 
        name = 'Untuned SVC with RBF Kernel'
    )
)
roc_svc_poly_tuned.add_shape(
        type = 'line', 
        x0 = 0, 
        y0 = 0, 
        x1 = 1, 
        y1 = 1, 
        line_color = 'Grey',
        line_width = 3,
        line_dash = 'dashdot'
    )
roc_svc_poly_tuned.update_layout(
    title = 'ROC Curves for Tuned and Untuned SVC models with Poly Kernel',
    title_x = 0.5,
    xaxis_title = 'False Positive Rate',
    yaxis_title = 'True Positive Rate', 
    height = 500, 
    width = 700
)

roc_svc_poly_tuned.show()

In [69]:
# Saving the best estimator for further development
svc_final = grid_poly_svc.best_estimator_

## Model Summary

Tuning hyperparameters for SVC with Polynomial Kernel resulted in higher AUC, accuracy, recall and precision metrics. Furthermore, final results are highly similar to those of SVC with RBF Kernel. Since the latter algorithm is costly to train, the former one seems like a promising model for final predictions. However, we can try improving it by applying Adaptive Boosting. In the end, we should dive into Random Forest Classifier before choosing the final model which will be tested on test data!

# Developing Ensemble Models

## Applying Adaptive Boosting

Even though SVC with Polynomial Kernel is a really good learner, we can try applying adaptive boosting to see whether predictions are going to be slightly better.

In [70]:
from sklearn.ensemble import AdaBoostClassifier
adacl = AdaBoostClassifier(base_estimator = svc_final, n_estimators = 100, learning_rate = 0.1, random_state = 123, algorithm = 'SAMME')
adacl.fit(X_train_std, y_train)
accuracy_score(y_train, adacl.predict(X_train_std))

0.8405700475039587

As we can see, accuracy is lower when applying adaptive boosting without cross-validation than with the base, cross-validated estimator. Therefore, we are going to keep the base estimator as it is.

## Developing Random Forest Classifier

Developing Decision Tree Classifier resulted in really good predictions. Therefore, we can use random forest classifier to improve those predictions even more!

In [71]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(
    max_depth = 20,
    n_estimators = 100, 
    criterion = 'gini', 
    bootstrap = True, 
    random_state = 123, 
    n_jobs = -1, 
    class_weight = 'balanced'
)

In [72]:
rfc_cross = cross_validate(
    rfc, 
    X_train_std, 
    y_train, 
    n_jobs = -1, 
    scoring = [
        'accuracy', 'precision', 'recall', 'roc_auc'
    ], 
    cv = 10
)

In [73]:
rfc_cross_metrics = pd.DataFrame(rfc_cross).iloc[:, 2:].mean()
rfc_cross_metrics

test_accuracy     0.988416
test_precision    0.990633
test_recall       0.960438
test_roc_auc      0.992798
dtype: float64

### Untuned Random Forest Classifier: Final Results from Cross-Validation

| Metric  | Value|
|-------- |------|
|Accuracy |0.988 |
|Precision|0.991 |
|Recall   |0.960 |
|AUC      |0.993 |

Now, let us save False Positive Rates and True Positive Rates for untuned model. Before plotting the ROC curve for this model, we should tune hyperparameters, and then present both ROC curves on the same plot, along with ROC curve for final SVC model.

In [74]:
y_scores = cross_val_predict(
    rfc, 
    X_train_std, 
    y_train, 
    cv = 10, 
    n_jobs = -1, 
    method = 'predict_proba'
)
y_scores = y_scores[:,1]

In [75]:
# Saving FPR and TPR 
fpr_rfc, tpr_rfc, thresholds_rfc = roc_curve(y_train, y_scores)

### Hyperparameter Tuning for Random Forest Classifier

In [76]:
rfc_params = {
    'n_estimators' : [100, 180, 260], 
    'max_depth' : [18, 20, 22, 25], 
    'max_features' : [0.1, 0.3, 0.5], 
    'min_samples_leaf' : [2, 5, 10]}

In [77]:
grid_rfc = GridSearchCV(
    RandomForestClassifier(
        criterion = 'gini', 
        bootstrap = True, 
        random_state = 123, 
        n_jobs = -1, 
        class_weight = 'balanced'
    ), 
    param_grid = rfc_params,     
    n_jobs = -1, 
    cv = 10, 
    scoring = [
        'roc_auc', 'accuracy', 'precision', 'recall'
    ],
    refit = 'roc_auc'
)

In [78]:
grid_rfc.fit(X_train_std, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True,
                                              class_weight='balanced',
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=-1,
                                              oob_score=False, random_state=123,
                                              verbose=0, warm_start=False),
            

In [79]:
final_rfc = grid_rfc.best_estimator_
print(final_rfc)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=22, max_features=0.5,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=2,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=180, n_jobs=-1, oob_score=False,
                       random_state=123, verbose=0, warm_start=False)


In [80]:
print('Best AUC score:', grid_rfc.best_score_)

Best AUC score: 0.9930788582647685


In [81]:
# Accessing the final metrics for the best estimator
rfc_metrics = pd.DataFrame(grid_rfc.cv_results_)
rfc_metrics.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_max_depth', 'param_max_features', 'param_min_samples_leaf',
       'param_n_estimators', 'params', 'split0_test_roc_auc',
       'split1_test_roc_auc', 'split2_test_roc_auc', 'split3_test_roc_auc',
       'split4_test_roc_auc', 'split5_test_roc_auc', 'split6_test_roc_auc',
       'split7_test_roc_auc', 'split8_test_roc_auc', 'split9_test_roc_auc',
       'mean_test_roc_auc', 'std_test_roc_auc', 'rank_test_roc_auc',
       'split0_test_accuracy', 'split1_test_accuracy', 'split2_test_accuracy',
       'split3_test_accuracy', 'split4_test_accuracy', 'split5_test_accuracy',
       'split6_test_accuracy', 'split7_test_accuracy', 'split8_test_accuracy',
       'split9_test_accuracy', 'mean_test_accuracy', 'std_test_accuracy',
       'rank_test_accuracy', 'split0_test_precision', 'split1_test_precision',
       'split2_test_precision', 'split3_test_precision',
       'split4_test_precision', 'split5_tes

In [82]:
rfc_metrics = rfc_metrics[
    ['params', 'mean_test_roc_auc', 'mean_test_accuracy', 'mean_test_precision', 'mean_test_recall']]
rfc_metrics['mean_metrics'] = svc_poly_metrics.iloc[:, 1:].mean(axis = 1)
rfc_metrics.head(3)

,params,mean_test_roc_auc,mean_test_accuracy,mean_test_precision,mean_test_recall,mean_metrics
0,"{'max_depth': 18, 'max_features': 0.1, 'min_sa...",0.990363,0.980415,0.975041,0.941887,0.931278
1,"{'max_depth': 18, 'max_features': 0.1, 'min_sa...",0.990609,0.980748,0.976476,0.941888,0.931278
2,"{'max_depth': 18, 'max_features': 0.1, 'min_sa...",0.990646,0.980915,0.977513,0.941538,0.931278


In [83]:
grid_rfc.best_params_

{'max_depth': 22,
 'max_features': 0.5,
 'min_samples_leaf': 2,
 'n_estimators': 180}

In [84]:
rfc_metrics.loc[rfc_metrics['params'] == grid_rfc.best_params_, :].iloc[:, 1:-1].T

,73
mean_test_roc_auc,0.993079
mean_test_accuracy,0.988249
mean_test_precision,0.992388
mean_test_recall,0.957990


### Tuned Decision Tree Classifier: Final Results from Cross-Validation

| Metric  | Value|
|-------- |------|
|Accuracy |0.988 |
|Precision|0.992 |
|Recall   |0.958 |
|AUC      |0.993 |

### ROC Curves for models - Final Comparison

In [85]:
y_scores = cross_val_predict(
    final_rfc, 
    X_train_std, 
    y_train, 
    cv = 10, 
    n_jobs = -1, 
    method = 'predict_proba'
)
y_scores = y_scores[:,1]

In [86]:
# Saving FPR and TPR for Tuned model
fpr_rfc_t, tpr_rfc_t, thresholds_rfc_t = roc_curve(y_train, y_scores)

In [87]:
# Comparing ROC Curves of Random Forest Classifier and SVC with Polynomial Kernel
final_roc = go.Figure()
final_roc.add_trace(
    go.Scatter(
        x = fpr_svc_poly_t, 
        y = tpr_svc_poly_t, 
        name = 'Tuned SVC with Poly Kernel'
    
    )
)
final_roc.add_trace(
    go.Scatter(
        x = fpr_rfc, 
        y = tpr_rfc, 
        name = 'Untuned Random Forest Classifier'
    )
)
final_roc.add_trace(
    go.Scatter(
        x = fpr_rfc_t, 
        y = tpr_rfc_t, 
        name = 'Tuned Random Forest Classifier'
    )
)
final_roc.add_shape(
        type = 'line', 
        x0 = 0, 
        y0 = 0, 
        x1 = 1, 
        y1 = 1, 
        line_color = 'Grey',
        line_width = 3,
        line_dash = 'dashdot'
    )
final_roc.update_layout(
    title = 'ROC Curves for Tuned and Untuned SVC models with Poly Kernel',
    title_x = 0.5,
    xaxis_title = 'False Positive Rate',
    yaxis_title = 'True Positive Rate', 
    height = 500, 
    width = 700
)

final_roc.show()

### Model Summary
We can see that random forest calssifier outperforms SVC with Polynomial Kernel. Furthermore, tuning hyperparameters of Random Forest Classifier did not lead to better metrics overall, since the metrics for untuned model were already highly satisfying.

We can conclude that the best model so far is Random Forest Classifier - it is fast to train and yields high accuracy on hold-out data (during cross-validation). Finally, we should test this model on test data and present the feature importances as well!

# Model Evaluation and Visualization

In [88]:
# Standardizing Test Data
X_test_std = X_test
X_test_std.iloc[:, :5] = stdscaler.fit_transform(X_test.iloc[:, :5])
X_test_std.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,salary_low,salary_medium,department_RandD,department_accounting,department_hr,department_management,department_marketing,department_product_mng,department_sales,department_support,department_technical
10627,1.471674,0.287163,-0.643887,-0.217436,-1.019102,0,0,0,1,0,0,0,0,0,0,1,0,0
2703,0.589496,1.217936,-0.643887,1.384943,1.017744,0,0,1,0,0,0,1,0,0,0,0,0,0
6059,0.348902,1.217936,-0.643887,-0.337615,0.338795,0,0,0,1,0,1,0,0,0,0,0,0,0
3258,-0.533277,-0.643610,-0.643887,0.423516,-0.340153,1,0,0,1,0,0,0,0,0,0,0,1,0
4565,-0.172386,1.043416,-0.643887,-1.158834,-1.019102,0,0,0,1,0,0,0,0,0,0,0,0,0


In [89]:
# Plotting the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
conf_matrix = confusion_matrix(y_test, final_rfc.predict(X_test_std))
heatmap_conf = pff.create_annotated_heatmap(
    np.flipud(conf_matrix), 
    x = ["Didn't leave the company", "Left the Company"],
    y = ["Left the company", "Didn't leave the company"],
    colorscale = 'blues'
)

heatmap_conf.update_layout(
    width = 600,
    height = 400,
    xaxis_title = 'Predicted',
    yaxis_title = 'Actual'
)

heatmap_conf.show()

In [90]:
# Printing out classification results
print(classification_report(y_test, final_rfc.predict(X_test_std)))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2286
           1       0.99      0.95      0.97       714

    accuracy                           0.99      3000
   macro avg       0.99      0.98      0.98      3000
weighted avg       0.99      0.99      0.99      3000



In [91]:
# Printing out feature importances
feature_importances = pd.DataFrame(
    {'Feature_name' : X_test_std.columns, 
     'Importance' : final_rfc.feature_importances_}
)
feature_importances.sort_values(
    by = 'Importance',
    ascending = False, 
    inplace = True
)
feature_importances

,Feature_name,Importance
0,satisfaction_level,0.321520
4,time_spend_company,0.272724
3,average_montly_hours,0.139457
2,number_project,0.123729
1,last_evaluation,0.117645
7,salary_low,0.004450
17,department_technical,0.003686
5,Work_accident,0.003216
15,department_sales,0.002992
8,salary_medium,0.002563


In [92]:
# Plotting Feature Importances
feature_plot = px.bar(
    data_frame = feature_importances.sort_values(by = 'Importance'), 
    x = 'Importance', 
    y = 'Feature_name', 
    labels = {'Feature_name' : 'Feature Names'}, 
    orientation = 'h',
    title = 'Feature Imortances for Predicting Turnover'
)
feature_plot.update_layout(title_x = 0.5)
feature_plot.show()

# Concluding Remarks

The goal of the project was to develop the model which would accurately predict the employee turnover and to gain insights into factors which drive the employee turnover the most!

The final model tested on the held-out dataset was Random Forest Classifier which achieved **99%** accuracy! Furthermore, we can conclude there are 5 main factors which are highly correlated with leaving the company. Those are:

1. Job Satisfaction (negative relationship)
2. Time Spent in Company (positive relationship)
3. Average Hours Worked Per Month (positive relationship)
4. Number of Projects (positive relationship)
5. Time Since Last Performance Evaluation (positive relationship)

The relationship between job satisfcation and leaving the company is straightforward - if employee is not satisfied with the current job, they are more likely to leave the company. However, the next two relationships could maybe come as a surprise. However, it is more likely that the employees who worked in the company for longer time and have more responsibilites are likely to leave the job. The reason is simple - new employees often give a company a shot for a few months or even years and they don't have that much responsibility in the beginning. The final two relationships are particularly interesting. It seems that the number of projects could overwhelm the employees, which could lead to reduction in job satisfaction and, ultimately, leaving the job. In the end, the relationship between time since last performance evaluation is negatively correlated with employee churn, which would mean that performance evaluation has some meaning for employees - e.g. they can test their performance, they have a feeling that company value their performance and works towards enhancing the performance of its employees, etc.

These results have several implications. First of all, collecting more data (if needed) should focus on these 5 factors, especially if gathering other information is time consuming. Second of all, this model was trained on a large set of data and can reliably be used for any future predictions. In the end, developing strategies for employee retention should focus on these 5 factors, with greater emphasis on *Job Satisfcation*. Further details, insights and recommendations are presented in a report.